In [8]:
import os

os.listdir('.')

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'CAMELYON dataset.pdf',
 'features.csv',
 'histopathologic-cancer-detection',
 'histopathologic-cancer-detection.zip',
 'solution.ipynb']

# Split up dataset

A key finding in this project has been the size of the dataset that I have to navigate every time I want to create a batch. Since most of the processing is done using a backend written in C, it appears to be quick. However, the time taken to iterate over a list of imaged and read them into arrays may be what is making this network take so long!

Therefore, I will be using the Keras `ImageDataGenerator` class! 

It needs the dataset to be split up by classes.

In [7]:
import pandas as pd
from shutil import copyfile

df = pd.read_csv('histopathologic-cancer-detection/train_labels.csv')
cancer = df[df['label'] == 1]
noncancer = df[df['label'] == 0]

for filename in cancer['id']:
    copyfile(
        'histopathologic-cancer-detection/train/{0}'.format(filename + '.tif'),
        'histopathologic-cancer-detection/train/cancer/{0}'.format(filename + '.tif')
    )

for filename in noncancer['id']:
    copyfile(
        'histopathologic-cancer-detection/train/{0}'.format(filename + '.tif'),
        'histopathologic-cancer-detection/train/noncancer/{0}'.format(filename + '.tif')
    )

# Write a batch generator

As per the keras documentation, the `flow_from_directory` function returns a tuple of (x, y) arrays. This is consistent with the format that can be used in the model!

In [2]:
import keras

image_generator = keras.preprocessing.image.ImageDataGenerator()

batch_generator = image_generator.flow_from_directory(
    'histopathologic-cancer-detection/split_dataset/',
    target_size=(96, 96),
    class_mode='binary',
    batch_size=64
)

Found 220025 images belonging to 2 classes.


# Define a model

I will attempt to define a model from scratch and train it on my GPU. It seems to perform quite well against the GPU benchmarks written in the keras examples so I have confidence. This message would not exist for long if I was not successful.

In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D
from keras import backend as K
from keras.backend import var, sum, min, max

import imageio
import numpy as np
import os
import pandas as pd

model = Sequential()
# Convolution layer 1
model.add(
    Conv2D(
        64,
        kernel_size=(5, 5),
        input_shape=(96, 96, 3),
        padding='same',
        activation='relu'
    )
)
model.add(BatchNormalization())

# Convolution layer 2
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # 48

# Convolution layer 3
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # 24

# Convolution layer 4
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # 12

# Convolution layer 5
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # 6

# Convolution layer 6
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # 3


# FC
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

model.fit_generator(
    batch_generator,
    steps_per_epoch=1000, 
    epochs=5
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1000/1000 [==============================] - 155s 155ms/step - loss: 0.4104 - acc: 0.8252
Epoch 2/5
1000/1000 [==============================] - 151s 151ms/step - loss: 0.2998 - acc: 0.8728
Epoch 3/5
1000/1000 [==============================] - 151s 151ms/step - loss: 0.2502 - acc: 0.8994
Epoch 4/5
1000/1000 [==============================] - 152s 152ms/step - loss: 0.2162 - acc: 0.9163
Epoch 5/5
1000/1000 [==============================] - 150s 150ms/step - loss: 0.1970 - acc: 0.9246


# Testing the dataset

In [ ]:
import numpy as np

